In [1]:
#import numpy and pandas
import numpy as np 
import pandas as pd

In [2]:
#import data set
titanic = pd.read_csv('train.csv')
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
#Confirming each PassengerId is unique in case there are double counts for things such as groupings of families
titanic['PassengerId'].is_unique

True

In [5]:
titanic['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [6]:
# survival rate of the titanic
titanic_survival_rate = (titanic[titanic['Survived']==1]['Survived'].sum())/(titanic['Survived'].count())
titanic_survival_rate_percentage = "{:.2%}".format(titanic_survival_rate)
titanic_survival_rate_percentage

'38.38%'

In [69]:
#What is the survival rate between male and female
data = titanic[['Survived','Sex']]
gender_survival_rate = pd.pivot_table(data,index='Sex', values='Survived', aggfunc=['sum','count']).reset_index()
gender_survival_rate.columns = gender_survival_rate.columns.map(' '.join)
gender_survival_rate = gender_survival_rate.rename(columns={'sum Survived':'Sum Survived', 'count Survived':'Count Survived'})
gender_survival_rate['Survival Rate'] = gender_survival_rate['Sum Survived']/gender_survival_rate['Count Survived']
gender_survival_rate['Survival Rate'] = gender_survival_rate['Survival Rate'].map("{:.2%}".format)
gender_survival_rate

Sex,Sum Survived,Count Survived,Survival Rate
female,233,314,74.20%
male,109,577,18.89%


In [66]:
#of the survivors, how many were underage(18) including the split between gender and their corresponding survival rate
data = titanic[['Survived','Sex']]
data['Underage'] = titanic['Age'] < 18
data['Underage'].replace([True,False],[0,1])
underage_survival_rate = pd.pivot_table(data, index='Sex', values=['Survived', 'Underage'], aggfunc=['sum','count']).reset_index()
underage_survival_rate.columns = underage_survival_rate.columns.map(' '.join)

underage_survival_rate['Ratio of Underage by Gender'] = underage_survival_rate['sum Underage']/underage_survival_rate['sum Survived']
underage_survival_rate['Ratio of Underage by Gender'] = underage_survival_rate['Ratio of Underage by Gender'].map("{:.2%}".format)

underage_survival_rate['Ratio of Underage by Overall Survived'] = underage_survival_rate['sum Underage']/underage_survival_rate['count Survived']
underage_survival_rate['Ratio of Underage by Overall Survived'] = underage_survival_rate['Ratio of Underage by Overall Survived'].map("{:.2%}".format)

underage_survival_rate = underage_survival_rate.drop(columns='count Underage')
underage_survival_rate = underage_survival_rate.style.hide_index()
underage_survival_rate

<ipython-input-66-386e2a823c53>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Underage'] = titanic['Age'] < 18


Sex,sum Survived,sum Underage,count Survived,Ratio of Underage by Gender,Ratio of Underage by Overall Survived
female,233,55,314,23.61%,17.52%
male,109,58,577,53.21%,10.05%


In [81]:
# # #What is the makeup for Pclass in the group that survived?
pclass_survival = titanic[['Pclass','Survived']]
pclass_survival = pclass_survival.groupby(['Pclass']).sum().reset_index().style.hide_index()
# pclass_survival = pclass_survival.groupby(['Pclass']).agg({'Survived':['sum','count']}).reset_index()
# pclass_survival.columns =pclass_survival.columns.map(' '.join)
pclass_survival


Pclass,Survived
1,136
2,87
3,119


In [102]:
#what is the gender make up and age_group make up for survivors in each pclass
#standard age grouping
# Under 12 years old.
# 12-17 years old.
# 18-24 years old.
# 25-34 years old.
# 35-44 years old.
# 45-54 years old.
# 55-64 years old.
# 65-74 years old.

data = titanic[['Pclass','Survived','Sex','Age']]
data = data[data['Survived'] == 1]

data['Under 12 Years Old'] = (data['Age'] < 12).replace([True,False],[1,0])
data['12-17 Years Old'] = ((data['Age'] >= 12) & (data['Age'] <= 17)).replace([True,False],[1,0])
data['18-24 Years Old'] = ((data['Age'] >= 18) & (data['Age'] <= 24)).replace([True,False],[1,0])
data['25-34 Years Old'] = ((data['Age'] >= 25) & (data['Age'] <= 34)).replace([True,False],[1,0])
data['35-44 Years Old'] = ((data['Age'] >= 35) & (data['Age'] <= 44)).replace([True,False],[1,0])
data['45-54 Years Old'] = ((data['Age'] >= 45) & (data['Age'] <= 54)).replace([True,False],[1,0])
data['55-64 Years Old'] = ((data['Age'] >= 55) & (data['Age'] <= 64)).replace([True,False],[1,0])
data['65-74 Years Old'] = ((data['Age'] >= 65) & (data['Age'] <= 74)).replace([True,False],[1,0])
data['Over 74 Years Old'] = (data['Age'] > 74).replace([True,False],[1,0])

data = data.drop(columns=['Survived','Age'])

# data.groupby()
data


,Pclass,Sex,Under 12 Years Old,12-17 Years Old,18-24 Years Old,25-34 Years Old,35-44 Years Old,45-54 Years Old,55-64 Years Old,65-74 Years Old,Over 74 Years Old
1,1,female,0,0,0,0,1,0,0,0,0
2,3,female,0,0,0,1,0,0,0,0,0
3,1,female,0,0,0,0,1,0,0,0,0
8,3,female,0,0,0,1,0,0,0,0,0
9,2,female,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
875,3,female,0,1,0,0,0,0,0,0,0
879,1,female,0,0,0,0,0,0,1,0,0
880,2,female,0,0,0,1,0,0,0,0,0
887,1,female,0,0,1,0,0,0,0,0,0
